In [1]:
from libs.chatglm3 import ChatGLM3
MODEL_PATH = "D:\Github\Repositories\models\huggingface\chatglm3-6b"
llm = ChatGLM3()
llm.load_model(MODEL_PATH)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [2]:
import torch
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.documents import Document
from langchain_community.llms import Tongyi

In [3]:

# llm = Tongyi()

In [4]:
from langchain.document_loaders import UnstructuredFileLoader
loader = UnstructuredFileLoader('./static/books/2022版中国乳腺癌随诊随访与健康管理指南.txt')
docs = loader.load()

In [5]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
# encode_kwargs = {'normalize_embeddings': False}
# embeddings = HuggingFaceEmbeddings(
#     model_name='shibing624/text2vec-base-chinese', encode_kwargs=encode_kwargs)
embeddings = HuggingFaceEmbeddings(model_name='shibing624/text2vec-base-chinese')

In [6]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=32)
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [7]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [8]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [9]:
response = retrieval_chain.invoke({"input": "子宫内膜增厚的管理有哪些方法?"})
print(response["answer"])


子宫内膜增厚的管理方法包括：

1. 内分泌治疗：对于使用内分泌药物（如他莫昔芬）治疗的患者，需要关注子宫内膜增厚的问题。针对绝经前患者，若子宫内膜厚度在月经周期的不同阶段存在显著差异，增殖期子宫内膜厚度≥12 mm，则判断为子宫内膜增厚。针对绝经后患者，若服用他莫昔芬的患者子宮内膜厚度≥8 mm，或服用AI或托瑞米芬的病人子宫内膜厚度≥4 mm，则判断为子宫内膜增厚。这些方法需要结合临床症状及经阴道超声综合判断子宫内膜增厚情况。

2. 随诊检测：对于已出现子宫内膜增厚的患者，应提高随访频率，建议每3~6个月进行检测。

3. 生活及性生活管理：健康及适度的性生活有利于乳腺癌患者的身心康复，但需要提醒患者注意避孕，推荐物理屏障避孕法，避免使用激素类药物避孕法。

4. 治疗：当子宫内膜检查结果异常时，首先应判断是否与内分泌药物（如他莫昔芬）相关，随后依据评估结果给予适当的治疗。


In [16]:
torch.cuda.empty_cache()
response = retrieval_chain.invoke({"input": "乳腺纤维化跟哪些基因有关?"})
print(response["answer"])


乳腺纤维化与TGFB1基因有关，同时伴有至少一个拷贝C⁃509T时，患者在保乳术后全乳放疗后发生乳腺纤维化的风险明显增加。


In [14]:
llm.history=[]